In [5]:
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import requests
import pymongo
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo
from webdriver_manager.chrome import ChromeDriverManager

In [7]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=True)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Driver [C:\Users\grace\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


### NASA Mars News

In [25]:
# scrap the News URL
news_url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
browser.visit(news_url)
html = browser.html
news_soup = BeautifulSoup(html, 'html.parser')

In [86]:
# Collect the latest News Title and Paragraph Text
news_title = news_soup.find_all('div', class_='content_title')[1].text
news_p = news_soup.find_all('div', class_='article_teaser_body')[0].text

print(news_title)
print("--------------------------------------------------------------------")
print(news_p)

NASA's Ingenuity Mars Helicopter Logs Second Successful Flight
--------------------------------------------------------------------
The small rotorcraft’s horizons were expanded on its second flight. 


### JPL Mars Space Images - Featured Image

In [92]:
import time

In [115]:
# scrap the JPL URL
images_url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'
browser.visit(images_url)
time.sleep(1)
html = browser.html
images_soup = BeautifulSoup(html, 'html.parser')

In [116]:
# Find featured image URL
relative_image_path = images_soup.find_all('a')[2]['href']
base_jpl_url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/'
featured_image_url = base_jpl_url + relative_image_path
print(featured_image_url)

https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars2.jpg


### Mars Facts

In [117]:
# scrap the Mars Facts URL
facts_url  = 'https://space-facts.com/mars/'
tables = pd.read_html(facts_url)
facts_df = tables[0].rename(columns={0:'Facts',1:'Data'}).set_index('Facts')
facts_df

,Data
Facts,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [118]:
facts_table = facts_df.to_html().replace('\n','')
print(facts_table)

<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Data</th>    </tr>    <tr>      <th>Facts</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <th>Moons:</th>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>Orbit Distance:</th>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <th>Orbit Period:</th>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>Surface Temperature:</th>      <td>-87 to -5 °C</td>    </tr>    <tr>      <th>First Record:</th>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>Recorded By:</th>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>


### Mars Hemispheres

In [119]:
# scrap the Mars hemisphere URL
hemi_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hemi_url)
hemi_html = browser.html
hemi_soup = BeautifulSoup(hemi_html, 'html.parser')

In [120]:
# Build the dictionary containing the title and image of each hemisphere
mars_hemispheres = hemi_soup.find('div', class_='collapsible results')
mars_hemi_items = mars_hemispheres.find_all('div', class_='item')
base_hemi_url = 'https://astrogeology.usgs.gov'
hemi_image_urls = []

# Use for loop to extract the URL for each hemisphere
for i in mars_hemi_items:
    # Collect Title
    hemisphere = i.find('div', class_='description')
    title = hemisphere.h3.text
    
    # Collect image URL
    hemisphere_link = hemisphere.a["href"]    
    browser.visit(base_hemi_url + hemisphere_link)
    
    image_html = browser.html
    image_soup = BeautifulSoup(image_html, 'html.parser')
    
    image_link = image_soup.find('div', class_='downloads')
    image_url = image_link.find('li').a['href']

    # Add the title and image info into a dictionary
    image_dict = {}
    image_dict['title'] = title
    image_dict['img_url'] = image_url
    
    hemi_image_urls.append(image_dict)

hemi_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

### Create a general dictionary containing all above information

In [121]:
mars_info_dict = {
        "news_title": news_title,
        "news_p": news_p,
        "featured_image_url": featured_image_url,
        "mars_fact": str(facts_table),
        "mars_hemisphere": hemi_image_urls
    }

mars_info_dict

{'news_title': "NASA's Ingenuity Mars Helicopter Logs Second Successful Flight",
 'news_p': 'The small rotorcraft’s horizons were expanded on its second flight. ',
 'featured_image_url': 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars2.jpg',
 'mars_fact': '<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Data</th>    </tr>    <tr>      <th>Facts</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <th>Moons:</th>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>Orbit Distance:</th>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <th>Orbit Period:</th>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>Surface Temperature:</th>      <td